# 📓 _LangChain_ Quickstart

In this quickstart you will create a simple LCEL Chain and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the RAG triad of groundedness, context relevance and answer relevance.

You'll also learn how to use feedbacks for guardrails, via filtering retrieved context.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/quickstart/langchain_quickstart.ipynb)

## Setup
### Add API keys
For this quickstart you will need Open AI and Huggingface keys

In [1]:
# ! pip install trulens_eval openai langchain langchain-openai langchain_community faiss-cpu bs4 tiktoken

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

### Import from LangChain and TruLens

In [3]:
# Imports main tools:
from trulens_eval import TruChain, Tru
tru = Tru()

# Imports from LangChain to build app
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

/opt/anaconda3/envs/snowday/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Load documents

In [6]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

### Create Vector Store

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents, embeddings)

### Create RAG

In [8]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/opt/anaconda3/envs/snowday/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### Send your first request

In [9]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique that breaks down complex tasks into smaller and simpler steps to enhance model performance. It involves transforming big tasks into manageable tasks and exploring multiple reasoning possibilities at each step. Task decomposition can be done using simple prompting, task-specific instructions, or by relying on an external classical planner for long-horizon planning.'

## Initialize Feedback Function(s)

In [10]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance)
    .on_input_output()
)
# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps__.context.first.invoke.rets[:].page_content.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.first.steps__.context.first.invoke.rets[:].page_content .


## Instrument chain for logging with TruLens

In [11]:
tru_recorder = TruChain(rag_chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

In [12]:
with tru_recorder as recording:
    llm_response = rag_chain.invoke("What is Task Decomposition?")

display(llm_response)

"Task Decomposition is a technique that breaks down complex tasks into smaller and simpler steps to enhance model performance. It involves transforming big tasks into manageable tasks and shedding light on the model's thinking process. Task decomposition can be done using simple prompting, task-specific instructions, or with human inputs."

## Use guardrails

In addition to making informed iteration, we can also directly use feedback results as guardrails at inference time. In particular, here we show how to use the context relevance score as a guardrail to filter out irrelevant context before it gets passed to the LLM. This both reduces hallucination and improves efficiency.

Below, you can see the TruLens feedback display of each context relevance chunk retrieved by our RAG.

In [13]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
get_feedback_result(last_record, 'context_relevance_with_cot_reasons')

,question,context,ret
0,What is Task Decomposition?,Fig. 1. Overview of a LLM-powered autonomous a...,0.8
1,What is Task Decomposition?,Fig. 10. A picture of a sea otter using rock t...,0.2
2,What is Task Decomposition?,(3) Task execution: Expert models execute on t...,0.7
3,What is Task Decomposition?,Fig. 6. Illustration of how Algorithm Distilla...,0.4


Wouldn't it be great if we could automatically filter out context chunks with relevance scores below 0.5?

We can do so with the TruLens guardrail, *WithFeedbackFilterDocuments*. All we have to do is use the method `of_retriever` to create a new filtered retriever, passing in the original retriever along with the feedback function and threshold we want to use.

In [27]:
from trulens_eval.guardrails.langchain import WithFeedbackFilterDocuments

# note: feedback function used for guardrail must only return a score, not also reasons
f_context_relevance_score = (
    Feedback(provider.context_relevance)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

filtered_retriever = WithFeedbackFilterDocuments.of_retriever(
        retriever=retriever,
        feedback=f_context_relevance_score,
        threshold=0.5
    )

rag_chain = (
    {"context": filtered_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

✅ In context_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance, input context will be set to __record__.app.first.steps__.context.first.invoke.rets[:].page_content .


Then we can operate as normal

In [15]:
tru_recorder = TruChain(rag_chain,
    app_id='Chain1_ChatApplication_Filtered',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

with tru_recorder as recording:
    llm_response = rag_chain.invoke("What is Task Decomposition?")

display(llm_response)

'Task decomposition is a technique that breaks down complex tasks into smaller and simpler steps to enhance model performance. It involves transforming big tasks into manageable tasks and exploring multiple reasoning possibilities at each step. Task decomposition can be done using simple prompting, task-specific instructions, or by relying on an external classical planner for long-horizon planning.'

## See the power of context filters!

If we inspect the context relevance of our retreival now, you see only relevant context chunks!

In [16]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
get_feedback_result(last_record, 'context_relevance_with_cot_reasons')

,question,context,ret
0,What is Task Decomposition?,Fig. 1. Overview of a LLM-powered autonomous a...,0.8
1,What is Task Decomposition?,(3) Task execution: Expert models execute on t...,0.7


## Retrieve records and feedback

In [17]:
# The record of the app invocation can be retrieved from the `recording`:

rec = recording.get() # use .get if only one record
# recs = recording.records # use .records if multiple

display(rec)

Record(record_id='record_hash_d1860d26b012e0d596b2a07c41c2fa24', app_id='Chain1_ChatApplication_Filtered', cost=Cost(n_requests=6, n_successful_requests=6, n_classes=0, n_tokens=5939, n_stream_chunks=0, n_prompt_tokens=5870, n_completion_tokens=69, cost=0.008934000000000001), perf=Perf(start_time=datetime.datetime(2024, 6, 10, 14, 59, 45, 831031), end_time=datetime.datetime(2024, 6, 10, 14, 59, 48, 429943)), ts=datetime.datetime(2024, 6, 10, 14, 59, 48, 429972), tags='-', meta=None, main_input='What is Task Decomposition?', main_output='Task decomposition is a technique that breaks down complex tasks into smaller and simpler steps to enhance model performance. It involves transforming big tasks into manageable tasks and exploring multiple reasoning possibilities at each step. Task decomposition can be done using simple prompting, task-specific instructions, or by relying on an external classical planner for long-horizon planning.', main_error=None, calls=[RecordAppCall(call_id='dabf440

In [18]:
# The results of the feedback functions can be rertireved from
# `Record.feedback_results` or using the `wait_for_feedback_result` method. The
# results if retrieved directly are `Future` instances (see
# `concurrent.futures`). You can use `as_completed` to wait until they have
# finished evaluating or use the utility method:

for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)

# See more about wait_for_feedback_results:
# help(rec.wait_for_feedback_results)

relevance 0.8
context_relevance_with_cot_reasons 0.75
groundedness_measure_with_cot_reasons 0.9666666666666667


In [22]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,groundedness_measure_with_cot_reasons,context_relevance_with_cot_reasons,relevance_calls,groundedness_measure_with_cot_reasons_calls,context_relevance_with_cot_reasons_calls,latency,total_tokens,total_cost
0,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_814101041a711b5490f062e8d8f98718,"""What is Task Decomposition?""","""Task Decomposition is a technique that breaks...",-,"{""record_id"": ""record_hash_814101041a711b5490f...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-06-10T14:59:30.184984"", ""...",2024-06-10T14:59:32.874274,0.8,1.000000,0.525,[{'args': {'prompt': 'What is Task Decompositi...,[{'args': {'source': ['Fig. 1. Overview of a L...,[{'args': {'question': 'What is Task Decomposi...,2,3302,0.004973
1,Chain1_ChatApplication_Filtered,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_d1860d26b012e0d596b2a07c41c2fa24,"""What is Task Decomposition?""","""Task decomposition is a technique that breaks...",-,"{""record_id"": ""record_hash_d1860d26b012e0d596b...","{""n_requests"": 6, ""n_successful_requests"": 6, ...","{""start_time"": ""2024-06-10T14:59:45.831031"", ""...",2024-06-10T14:59:48.429972,0.8,0.966667,0.750,[{'args': {'prompt': 'What is Task Decompositi...,[{'args': {'source': ['Fig. 1. Overview of a L...,[{'args': {'question': 'What is Task Decomposi...,2,5939,0.008934


In [21]:
tru.get_leaderboard(app_ids=[])

,groundedness_measure_with_cot_reasons,relevance,context_relevance_with_cot_reasons,latency,total_cost
app_id,,,,,
Chain1_ChatApplication,1.000000,0.8,0.525,2.0,0.004973
Chain1_ChatApplication_Filtered,0.966667,0.8,0.750,2.0,0.008934


## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.4.206:7876



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Learn more about the call stack

In [23]:
json_like = last_record.layout_calls_as_app()

In [24]:
json_like

Munch({'record_id': 'record_hash_d1860d26b012e0d596b2a07c41c2fa24', 'app_id': 'Chain1_ChatApplication_Filtered', 'cost': {'n_requests': 6, 'n_successful_requests': 6, 'n_classes': 0, 'n_tokens': 5939, 'n_stream_chunks': 0, 'n_prompt_tokens': 5870, 'n_completion_tokens': 69, 'cost': 0.008934000000000001}, 'perf': {'start_time': '2024-06-10T14:59:45.831031', 'end_time': '2024-06-10T14:59:48.429943'}, 'ts': '2024-06-10T14:59:48.429972', 'tags': '-', 'meta': None, 'main_input': 'What is Task Decomposition?', 'main_output': 'Task decomposition is a technique that breaks down complex tasks into smaller and simpler steps to enhance model performance. It involves transforming big tasks into manageable tasks and exploring multiple reasoning possibilities at each step. Task decomposition can be done using simple prompting, task-specific instructions, or by relying on an external classical planner for long-horizon planning.', 'main_error': None, 'calls': [{'call_id': 'dabf4400-dc97-4353-914d-d1ea

In [25]:
from ipytree import Tree, Node

def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node('Record ID: {}'.format(data['record_id'])))
    tree.add_node(Node('App ID: {}'.format(data['app_id'])))
    tree.add_node(Node('Cost: {}'.format(data['cost'])))
    tree.add_node(Node('Performance: {}'.format(data['perf'])))
    tree.add_node(Node('Timestamp: {}'.format(data['ts'])))
    tree.add_node(Node('Tags: {}'.format(data['tags'])))
    tree.add_node(Node('Main Input: {}'.format(data['main_input'])))
    tree.add_node(Node('Main Output: {}'.format(data['main_output'])))
    tree.add_node(Node('Main Error: {}'.format(data['main_error'])))
    
    calls_node = Node('Calls')
    tree.add_node(calls_node)
    
    for call in data['calls']:
        call_node = Node('Call')
        calls_node.add_node(call_node)
        
        for step in call['stack']:
            step_node = Node('Step: {}'.format(step['path']))
            call_node.add_node(step_node)
            if 'expanded' in step:
                expanded_node = Node('Expanded')
                step_node.add_node(expanded_node)
                for expanded_step in step['expanded']:
                    expanded_step_node = Node('Step: {}'.format(expanded_step['path']))
                    expanded_node.add_node(expanded_step_node)
    
    return tree

# Usage
tree = display_call_stack(json_like)
tree

Tree(nodes=(Node(name='Record ID: record_hash_d1860d26b012e0d596b2a07c41c2fa24'), Node(name='App ID: Chain1_Ch…